# EXPLOTATION ZONE

1. Load data
2. Integrate data
3. Perform concrete data quality 

## 1. Load data from trusted zone


In [1]:
!pip install duckdb

import duckdb
import pandas as pd

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/explotation/explotation.duckdb', read_only=False)

con = duckdb.connect(database='./explotation/explotation.duckdb', read_only=False)


df_governance = pd.read_sql("SELECT * FROM Governance_Data", con)
df_olympics = pd.read_sql("SELECT * FROM Olympics_Data", con)
df_countrylevel = pd.read_sql("SELECT * FROM Country_Level_Data", con)

In [4]:
df_governance.head()

,Country Code,year,CC,G,PV,RL,RQ,VA
0,AFG,1996,-1.291705,-2.175167,-2.417310,-1.788075,-2.090330,-1.908540
1,AFG,2000,-2.175167,-2.417310,-1.788075,-2.090330,-1.908540,-1.271724
2,AFG,2004,-2.417310,-1.788075,-2.090330,-1.908540,-1.271724,-2.173946
3,AFG,2008,-1.788075,-2.090330,-1.908540,-1.271724,-2.173946,-2.438969
4,AFG,2012,-2.090330,-1.908540,-1.271724,-2.173946,-2.438969,-1.780661


In [5]:
df_olympics.head()

,year,city,discipline,sport_group,ioc_code,iso_code,country,region,event_sex,participant_type,event,medal,olympic_type
0,1992,Barcelona,Diving,Swimming and Diving,CHN,CHN,China,Asia,Men,Single,10M Platform,Bronze,summer
1,1992,Barcelona,Diving,Swimming and Diving,USA,USA,USA,Americas,Women,Single,10M Platform,Bronze,summer
2,1992,Barcelona,Diving,Swimming and Diving,EUN,EUN,Unified Team,noregion,Men,Single,3M Springboard,Bronze,summer
3,1992,Barcelona,Diving,Swimming and Diving,GER,DEU,Germany,Europe,Women,Single,3M Springboard,Bronze,summer
4,1992,Barcelona,Swimming,Swimming and Diving,USA,USA,USA,Americas,Men,Single,100M Backstroke,Bronze,summer


In [6]:
df_countrylevel.head()

,year,city,iso,country,region,olympic_type,population,GDP_constant_2010_USD,land_area_km_sq,notes,population_%,GDP_%,land_area_%
0,1992,Barcelona,ALB,Albania,Europe,summer,3247039.0,4.183027e+09,27400,None,0.000614,0.000110,0.000248
1,1992,Barcelona,DZA,Algeria,Africa,summer,27181094.0,9.252182e+10,2381740,None,0.005142,0.002430,0.021531
2,1992,Barcelona,ASM,American Samoa,Oceania,summer,49593.0,3.762957e+08,200,None,0.000009,0.000010,0.000002
3,1992,Barcelona,AND,Andorra,Europe,summer,58888.0,2.018601e+09,470,None,0.000011,0.000053,0.000004
4,1992,Barcelona,AGO,Angola,Africa,summer,12968345.0,2.945149e+10,1246700,None,0.002453,0.000773,0.011270


## 2.2. Integrate with data governance


## 2.2.2. integrate dataframes

In [7]:
df_countrylevel['year'] = df_countrylevel['year'].astype(int)
df_governance['year'] = df_governance['year'].astype(int)

data = pd.merge(df_countrylevel, df_governance, left_on=['year', 'iso'], right_on=['year', 'Country Code'])
data.drop(columns=['Country Code'], inplace=True)

In [8]:
data.drop(columns=['notes', 'population_%', 'GDP_%', 'land_area_%'], inplace=True)

# Store data

In [9]:
con.execute("DROP TABLE IF EXISTS Governance_Data;")
con.execute("DROP TABLE IF EXISTS Olympics_Data;")
con.execute("DROP TABLE IF EXISTS Country_Level_Data;")

con.execute("DROP TABLE IF EXISTS Olympics_Data;")
con.execute("CREATE TABLE IF NOT EXISTS Olympics_Data AS SELECT * FROM df_olympics")

con.execute("DROP TABLE IF EXISTS Country_Data")
con.execute("CREATE TABLE IF NOT EXISTS Country_Data AS SELECT * FROM data")

con.execute("SHOW TABLES").fetchall()

[('Country_Data',), ('Olympics_Data',)]

In [10]:
con.close()